In [3]:
#@title Install Requirements
!pip install -qU "langchain[google-vertexai]"
!pip install -qU langchain-google-vertexai
!pip install langchain-google-genai
!pip install -qU langchain-core
!pip install -qU pypdf langchain_community
!pip install playwright
!playwright install

Playwright Host validation warning: 
╔══════════════════════════════════════════════════════╗
║ Host system is missing dependencies to run browsers. ║
║ Missing libraries:                                   ║
║     libwoff2dec.so.1.0.2                             ║
║     libgstgl-1.0.so.0                                ║
║     libgstcodecparsers-1.0.so.0                      ║
║     libavif.so.13                                    ║
║     libharfbuzz-icu.so.0                             ║
║     libenchant-2.so.2                                ║
║     libsecret-1.so.0                                 ║
║     libhyphen.so.0                                   ║
║     libmanette-0.2.so.0                              ║
╚══════════════════════════════════════════════════════╝
    at validateDependenciesLinux (/usr/local/lib/python3.11/dist-packages/playwright/driver/package/lib/server/registry/dependencies.js:216:9)
    at async Registry._validateHostRequirements (/usr/local/lib/python3.11/dist

In [ ]:
#@title Get Google AI API Key
## You need to put your Google AI API key.

import getpass
import os

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

Enter your Google AI API key: ··········


In [ ]:
#@title Authenticate with Google Cloud and your project ID
# Please put your google cloud project id.

import vertexai
from google.colab import auth

gcp_project_id = '' # @param {type: "string"}       ## PUT GOOGLE CLOUD ASSIGNMENT NAME HERE

auth.authenticate_user(project_id=gcp_project_id)

vertexai.init(project=gcp_project_id)

In [ ]:
#@title RAG Train Model
#!pip install -qU "langchain[google-vertexai]"
#!pip install -qU langchain-google-vertexai
#!pip install langchain-google-genai
#!pip install -qU langchain-core
#!pip install -qU pypdf langchain_community
#!pip install playwright
#!playwright install

import requests
import textwrap
from langchain.chat_models import init_chat_model
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders import AsyncChromiumLoader
from langchain_community.document_transformers import BeautifulSoupTransformer
# embed the document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_google_vertexai import VertexAIEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore
import urllib.request
import pandas as pd
from langchain_core.documents import Document
from langchain_community.document_loaders import PyPDFLoader
# Wrap in LangChain LLM
from langchain_community.llms import HuggingFacePipeline as LC_HF_Pipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.llms import HuggingFacePipeline

################################# PUT QUESTION HERE #######################################
question = "What is the rating for Mr. Peabody & Sherman?"


############################## FILES TO RAG TRAIN BELOW ###################################
# 1: Load movies_data_1.txt file
file_path = "./movies_data_1.txt"
txt_loader = TextLoader(file_path)
txt_doc_1 = txt_loader.load()

# 2: Load netflix_2025_released_movies CSV file
csv_2025_released_movies = pd.read_csv("./netflix_2025_released_movies.csv")

csv_doc_2 = []
for _, row in csv_2025_released_movies.iterrows():
    content = f"Title: {row.get('Title', '')}\nRelease Date: {row.get('Release Date', '')}\nGenre: {row.get('genre', '')}\nDirector(s): {row.get('Director(s)', '')}\nCast Highlights: {row.get('Cast Highlights', '')}Description: {row.get('Description', '')}\n"
    csv_doc_2.append(Document(page_content=content))

# 3: Load netflix_2024 pdf
pdf_path = "./netflix_2024.pdf"
pdf_loader = PyPDFLoader(pdf_path)
pdf_docs_3 = pdf_loader.load()

# 4: Load april_2025_movies pdf
pdf_path = "./netflix_april_2025.pdf"
pdf_loader = PyPDFLoader(pdf_path)
pdf_docs_4 = pdf_loader.load()

# 5: Load march_2025_movies pdf
pdf_path = "./netflix_march_2025.pdf"
pdf_loader = PyPDFLoader(pdf_path)
pdf_docs_5 = pdf_loader.load()

# 6: Load netflix_movies_1_to_100 CSV file
csv_movies_1_to_100 = pd.read_csv("./netflix_movies_1_to_100.csv")

csv_doc_6 = []
for _, row in csv_movies_1_to_100.iterrows():
    content = f"Rank: {row.get('Rank', '')}\nTitle: {row.get('Title', '')}\nRelease Year: {row.get('Year', '')}\nRotten Tomato Score (Rating): {row.get('Tomatometer', '')}\nCritics Consensus: {row.get('Critics Consensus', '')}\nSynopsis: {row.get('Synopsis', '')}\nCast: {row.get('Starring', '')}\nDirected By: {row.get('Directed By', '')}\nAvailable on: {row.get('Streaming On', '')}\n"
    csv_doc_6.append(Document(page_content=content))

csv_movies_ratings = pd.read_csv("./custom (1).csv")

csv_doc_7 = []
for _, row in csv_movies_ratings.iterrows():
    content = f"Title ID: {row.get('tconst', '')}\nPrimary Title: {row.get('primaryTitle', '')}\nOriginal Title: {row.get('originalTitle', '')}\nRelease Year: {row.get('startYear', '')}\nGenres: {row.get('genres', '')}\nAverage Rating: {row.get('averageRating', '')}\nNumber of Votes: {row.get('numVotes', '')}\n"
    csv_doc_7.append(Document(page_content=content))

csv_oscar_nominations = pd.read_csv("./custom_ocar (1).csv") 

csv_doc_8 = []
for _, row in csv_oscar_nominations.iterrows():
    content = f"Film Year: {row.get('year_film', '')}\nCeremony Year: {row.get('year_ceremony', '')}\nCeremony Number: {row.get('ceremony', '')}\nCategory: {row.get('category', '')}\nCanonical Category: {row.get('canon_category', '')}\nNominee: {row.get('name', '')}\nFilm: {row.get('film', '')}\nWinner: {'Yes' if row.get('winner', False) else 'No'}\n"
    csv_doc_8.append(Document(page_content=content))

##############################################################################################

# Combine all docs
docs = txt_doc_1 + csv_doc_2 + pdf_docs_3 + pdf_docs_4 +pdf_docs_5 + csv_doc_6 + csv_doc_7 + csv_doc_8


# Split the text into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)


# load chat model
llm = init_chat_model("gemini-2.0-flash-001", model_provider="google_vertexai")


# load embedding model
embedding_model = VertexAIEmbeddings(model="text-embedding-004")

# vector store
vector_store = InMemoryVectorStore(embedding=embedding_model)
print(f"Number of chunks to embed: {len(splits)}")
from tqdm import tqdm
vector_store.add_texts([docs.page_content for docs in tqdm(splits)])
#vector_store.add_texts([docs.page_content for docs in splits])
retriever = vector_store.as_retriever()

# Run without RAG
baseline_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an LLM that should answer questions to the best of your ability."),
    ("human", "{input}"),
])
baseline_chain = baseline_prompt | llm

baseline_response = baseline_chain.invoke({"input": question})

print("\n--- Pre-RAG Output ---")
print(textwrap.fill(baseline_response.content, width=110))



# run inference with RAG
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

system_prompt = (
    "You are an LLM that should answer my questions with correct answers."
    "Please answer the quuestion to the best of your ability. Use both the context and your own knowledge "
    "to answer the question as best you can.\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

results = rag_chain.invoke({"input": question})
output_text = results['answer']
wrapped_text = textwrap.fill(output_text, width=110)
print("\n--- Post-RAG Output ---")
print(wrapped_text)

#print(results['answer'])

Number of chunks to embed: 7275


100%|██████████| 7275/7275 [00:00<00:00, 1542881.21it/s]



--- Pre-RAG Output ---
*Mr. Peabody & Sherman* has the following ratings:  *   **IMDb:** 6.8/10 *   **Rotten Tomatoes:** 79% *
**Common Sense Media:** 3/5

--- Post-RAG Output ---
Mr. Peabody & Sherman has a Rotten Tomato Score of 81%. The audience rating on IMDB is 6.7. The age rating is
PG.


In [15]:
question = "Help me find a comedy with Adam Sandler on Netflix"
results = rag_chain.invoke({"input": question})
print(results['answer'])

Okay, here are the comedies with Adam Sandler that are available on Netflix, according to the information I have:

*   **Sandy Wexler** (2017)
*   **Adam Sandler: 100% Fresh** (2018)
*   **Anger Management** (2003)

I also found a listing for "**ADAM SANDLER 100% FRESH**" (2018) which appears to be the same as "**Adam Sandler: 100% Fresh**" (2018).


In [ ]:
results = rag_chain.invoke({"input": "Can you give me a list of Netflix romance movies with time travel?"})
print(results['answer'])

Here is a list of Netflix romance movies with time travel:
*   The Time Traveler's Wife
*   About Time
*   When We First Met
*   Kate & Leopold
